In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
num_epochs = 200
batch_size = 128
learning_rate = 3e-3

In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [4]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(113, 226),nn.ReLU(True),
            nn.Linear(226, 100),nn.ReLU(True), 
            nn.Linear(100, 10)
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 100),nn.ReLU(True),
            nn.Linear(100, 226),nn.ReLU(True),
            nn.Linear(226, 113),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded
    

In [5]:
model = autoencoder()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [6]:
import numpy as np
dataset=np.load('../data/all/all_IO_noleave_N.npz')
TrainX=np.asarray(np.asmatrix(dataset['TrainX'])[0,0].todense()).astype(np.float32)
ValidX=np.asarray(np.asmatrix(dataset['ValidX'])[0,0].todense()).astype(np.float32)
TestX=np.asarray(np.asmatrix(dataset['TestX'])[0,0].todense()).astype(np.float32)
TrainXm=np.asmatrix(dataset['TrainX'])[0,0].todense().astype(np.float32)
ValidXm=np.asmatrix(dataset['ValidX'])[0,0].todense().astype(np.float32)
TestXm=np.asmatrix(dataset['TestX'])[0,0].todense().astype(np.float32)

In [7]:
n=226
TrainXae=torch.from_numpy(np.vstack((TrainX[:,:int(n/2)],TrainX[:,int(n/2):]))).float()
ValidXae=torch.from_numpy(np.vstack((ValidX[:,:int(n/2)],ValidX[:,int(n/2):]))).float()
TestXae=torch.from_numpy(np.vstack((TestX[:,:int(n/2)],TestX[:,int(n/2):]))).float()

In [8]:
dataloader=DataLoader(TrainXae, batch_size=batch_size, shuffle=True)

In [ ]:
for epoch in range(num_epochs):
    for data in dataloader:
        encoding, recon = model(data)
        loss = criterion(recon, data)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
#     print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 1 == 0:
        encoding, recon = model(TrainXae)
        loss = criterion(recon, TrainXae)
        print('At epoch [{}/{}], Total Training Set loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data[0]))        

In [ ]:
encodingTrain, reconTrain = model(TrainXae)
lossTrain = criterion(reconTrain, TrainXae)
print('Training Set Draft Reconstruction Loss:{:.4f}'.format(lossTrain.data[0]))     
encodingValid, reconValid = model(ValidXae)
lossValid = criterion(reconValid, ValidXae)
print('Validation Set Draft Reconstruction Loss:{:.4f}'.format(lossValid.data[0]))
encodingTest, reconTest = model(TestXae)
lossTest = criterion(reconTest, TestXae)
print('Test Set Draft Reconstruction Loss:{:.4f}'.format(lossTest.data[0]))